## Imports and Constants

In [75]:
import os
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
os.environ["KMP_AFFINITY"] = "noverbose"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import json
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
tf.autograph.set_verbosity(3)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import utils

In [65]:
DATAPATH = '../data/dataset/'
MODELPATH = os.path.join(DATAPATH, 'humans/')
CLOTHINGPATH = os.path.join(DATAPATH, 'tops/')

## Load Data
### Load the humans

In [66]:
humans = pd.read_json(os.path.join(MODELPATH, 'model.json'))

In [67]:
humans

,Alex,Amanda,Anna,Anna Maria,Anneliese,Bree,Charlotte,Chesska,Chelsea,Clementine,...,Rosaline,Shadae,Shanaya,Sophia,Sophie,Stacey,Stefania,Summer,Thais,Tia
height,174.0,178.0,175.0,179.0,178.0,180.0,169.0,173.0,177.0,175.0,...,175.0,177.0,178.0,172.0,176.0,169.0,172.0,176.0,178.0,176.0
bust,81.0,86.0,84.0,84.0,84.0,99.0,88.0,89.0,84.0,86.0,...,96.0,86.0,96.0,100.0,113.0,101.0,97.0,84.0,86.5,104.0
waist,64.0,67.0,72.0,64.0,64.0,72.0,67.0,69.0,69.0,65.0,...,80.0,66.0,81.0,87.0,104.0,80.0,73.0,64.0,61.0,92.0
hips,91.0,89.0,98.0,90.0,91.0,107.0,93.0,94.0,96.0,91.0,...,107.0,91.0,107.0,109.0,132.0,98.0,103.0,96.0,92.5,111.0
url,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://images.prismic.io/birdsnest-blog/0f008...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...


In [68]:
humans = humans.transpose()

In [69]:
humans

,height,bust,waist,hips,url
Alex,174.0,81.0,64.0,91.0,https://birdsnest-images.imgix.net/system/imag...
Amanda,178.0,86.0,67.0,89.0,https://birdsnest-images.imgix.net/system/imag...
Anna,175.0,84.0,72.0,98.0,https://birdsnest-images.imgix.net/system/imag...
Anna Maria,179.0,84.0,64.0,90.0,https://birdsnest-images.imgix.net/system/imag...
Anneliese,178.0,84.0,64.0,91.0,https://images.prismic.io/birdsnest-blog/0f008...
...,...,...,...,...,...
Stacey,169.0,101.0,80.0,98.0,https://birdsnest-images.imgix.net/system/imag...
Stefania,172.0,97.0,73.0,103.0,https://birdsnest-images.imgix.net/system/imag...
Summer,176.0,84.0,64.0,96.0,https://birdsnest-images.imgix.net/system/imag...
Thais,178.0,86.5,61.0,92.5,https://birdsnest-images.imgix.net/system/imag...


In [79]:
humans.index

Index(['Alex', 'Amanda', 'Anna', 'Anna Maria', 'Anneliese', 'Bree',
       'Charlotte', 'Chesska', 'Chelsea', 'Clementine', 'Denniel', 'Edith',
       'Emma_1', 'Emma_2', 'Fiona', 'Gemma', 'Georgia', 'Grazi', 'Helen',
       'Jess', 'Jira', 'Jude', 'Kate', 'Katie', 'Lara', 'Laura', 'Lauren',
       'Lisa', 'Ljubenka', 'Louise', 'Lucy', 'Maddy_1', 'Maddy_2', 'Madeline',
       'Mae-Lee', 'Mandy', 'Marnee', 'Mercy', 'Miranda', 'Monique', 'Nadine',
       'Natalia', 'Natalie_1', 'Natalie_2', 'Nathalie_1', 'Nathalie_2',
       'Nikki', 'Olivia', 'Palatina', 'Perrie', 'Riina', 'Rosaline', 'Shadae',
       'Shanaya', 'Sophia ', 'Sophie', 'Stacey', 'Stefania', 'Summer', 'Thais',
       'Tia'],
      dtype='object')

### Download the humans

In [81]:
image_names = humans.index.to_list()
image_urls = humans.url.to_list()
download_path = os.path.join(MODELPATH, 'images')

if not os.path.exists(download_path):
    print(os.path.exists(download_path))
    os.makedirs(download_path)

# todo: modularize this, transfer to a utils file
for name, url in zip(image_names, image_urls):
    try: 
        response = requests.get(url, stream=True)
        response.raise_for_status()

        filename = os.path.join(download_path, name+'.png')

        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {name}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to download {name}")

False
Downloaded Alex
Downloaded Amanda
Downloaded Anna
Downloaded Anna Maria
Downloaded Anneliese
Downloaded Bree
Downloaded Charlotte
Downloaded Chesska
Downloaded Chelsea
Downloaded Clementine
Downloaded Denniel
Downloaded Edith
Downloaded Emma_1
Downloaded Emma_2
Downloaded Fiona
Downloaded Gemma
Downloaded Georgia
Downloaded Grazi
Downloaded Helen
Downloaded Jess
Downloaded Jira
Downloaded Jude
Downloaded Kate
Downloaded Katie
Downloaded Lara
Downloaded Laura
Downloaded Lauren
Downloaded Lisa
Downloaded Ljubenka
Downloaded Louise
Downloaded Lucy
Downloaded Maddy_1
Downloaded Maddy_2
Downloaded Madeline
Downloaded Mae-Lee
Downloaded Mandy
Downloaded Marnee
Downloaded Mercy
Downloaded Miranda
Downloaded Monique
Downloaded Nadine
Downloaded Natalia
Downloaded Natalie_1
Downloaded Natalie_2
Downloaded Nathalie_1
Downloaded Nathalie_2
Downloaded Nikki
Downloaded Olivia
Downloaded Palatina
Downloaded Perrie
Downloaded Riina
Downloaded Rosaline
Downloaded Shadae
Downloaded Shanaya
Downlo

In [56]:
# save it as a csv file
fp = os.path.join(MODELPATH, 'humans.csv')
humans.to_csv(fp, index=False)

### Load the tops
#### Load garment measurements and store in dataframe

In [57]:
# testing
path = '../data/dataset/tops/garment_measurement/2-in-1-jumper#Navy_meta.txt'

# path = 'dataset/tops/garment_measurement/natural-textured-poncho#Deeprain_meta.txt'

utils.parse_json(path)

[{'name': '2-in-1-jumper#Navy_meta',
  'size': 'M',
  'Bust': '120cm',
  'Waist': '118cm',
  'Hip': '126cm',
  'Length - shoulder to hem': '71-75cm',
  'Sleeve Length - neck to hem': '65cm'}]

In [59]:
dirpath = os.path.join(CLOTHINGPATH, 'garment_measurement')

data = []
count = 0
not_parsed = []

for file in os.listdir(dirpath):
    if file.endswith('.txt'):
        filepath = os.path.join(dirpath, file)

    data_ = utils.parse_json(filepath)
    if data_ is None:
        count += 1
        not_parsed.append(filepath)
        continue
    
    for d in data_:
        data.append(d)

Error parsing avola-tunic#Sky_meta.txt
Error parsing alskling-print-top#SeagullPr_meta.txt
Error parsing cambridge-henley#Khaki_meta.txt
Error parsing sail-away-upf-50-top#BlkWhtStrp_meta.txt
Error parsing lounge-tunic#Charcoal_meta.txt
Error parsing ulverstone-sweater#Black_meta.txt
Error parsing timberland-cardi-1#Bark_meta.txt
Error parsing burleigh-sweater#BlueStripe_meta.txt
Error parsing alexandria-long-cardi#Indigo_meta.txt
Error parsing merino-wrap-3#Navy_meta.txt
Error parsing rustler-shirt#Blush_meta.txt
Error parsing goddess-upf-50-tunic#WhtStripe_meta.txt
Error parsing leende-blouse#Abstract_meta.txt
Error parsing huddle-up-knit-1#Oatmeal_meta.txt
Error parsing swiss-spot-knit#TanWhite_meta.txt
Error parsing alexandria-long-cardi#Charcoal_meta.txt
Error parsing moulin-gauge-pullover#SeaCombo_meta.txt
Error parsing merino-wool-top-with-zips-1#Black_meta.txt
Error parsing momo-top#Black_meta.txt
Error parsing kendall-sweater#Khaki_meta.txt
Error parsing saffron-knit#Terracott

In [30]:
print(f"Number of files not parsed: {count}")

Number of files not parsed: 126


In [82]:
tops = pd.DataFrame(data)
tops

,name,size,Length - shoulder to hem,Sleeve Length - neck to hem,Hip,Bust,Waist,Length - waist to hem
0,natural-textured-poncho#Deeprain_meta,S,72cm,NaN,NaN,NaN,NaN,NaN
1,natural-textured-poncho#Deeprain_meta,M,73cm,NaN,NaN,NaN,NaN,NaN
2,natural-textured-poncho#Deeprain_meta,L,74cm,NaN,NaN,NaN,NaN,NaN
3,natural-textured-poncho#Deeprain_meta,XL,75cm,NaN,NaN,NaN,NaN,NaN
4,natural-textured-poncho#Deeprain_meta,XXL,76cm,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3823,the-flutter-sleeve-top#Denim_meta,14,75cm,NaN,131cm,112cm,NaN,NaN
3824,the-flutter-sleeve-top#Denim_meta,16,78cm,NaN,137cm,118cm,NaN,NaN
3825,the-flutter-sleeve-top#Denim_meta,18,79cm,NaN,143cm,124cm,NaN,NaN
3826,the-flutter-sleeve-top#Denim_meta,20,80cm,NaN,149cm,130cm,NaN,NaN


In [62]:
tops = pd.DataFrame(data)
tops

,name,size,Length - shoulder to hem,Sleeve Length - neck to hem,Hip,Bust,Waist,Length - waist to hem
0,natural-textured-poncho#Deeprain_meta,S,72cm,NaN,NaN,NaN,NaN,NaN
1,natural-textured-poncho#Deeprain_meta,M,73cm,NaN,NaN,NaN,NaN,NaN
2,natural-textured-poncho#Deeprain_meta,L,74cm,NaN,NaN,NaN,NaN,NaN
3,natural-textured-poncho#Deeprain_meta,XL,75cm,NaN,NaN,NaN,NaN,NaN
4,natural-textured-poncho#Deeprain_meta,XXL,76cm,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3823,the-flutter-sleeve-top#Denim_meta,14,75cm,NaN,131cm,112cm,NaN,NaN
3824,the-flutter-sleeve-top#Denim_meta,16,78cm,NaN,137cm,118cm,NaN,NaN
3825,the-flutter-sleeve-top#Denim_meta,18,79cm,NaN,143cm,124cm,NaN,NaN
3826,the-flutter-sleeve-top#Denim_meta,20,80cm,NaN,149cm,130cm,NaN,NaN


In [63]:
tops.describe()

,name,size,Length - shoulder to hem,Sleeve Length - neck to hem,Hip,Bust,Waist,Length - waist to hem
count,3828,3828,3737,3067,2953,3702,2540,6
unique,897,27,177,101,170,148,140,6
top,the-cotton-bamboo-tank#DarkNavy_meta,12,67cm,70cm,110cm,120cm,110cm,51cm
freq,9,610,190,157,79,117,91,1


In [ ]:

#todo: extract images from the given directory
#todo: extract model measurements
#todo: extract meta_descriptions

#todo?: combine the model measurements with the garment measurements to create a combined dataset
#todo?: think of processing the meta_descriptions

#### Download tops